In [1]:
from cvrpy.pso import PSO

pso = PSO(
    lambda vec: 1/sum(vec),
    num_iterations=100,
    num_particles=100,
    num_dimensions=20
)

solution = pso.optimize()

print(type(solution))
print(solution.position)


<class 'cvrpy.pso.Particle'>
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [1]:
from loggibud.v1.types import CVRPInstance
from cvrpy.utils import linear_distance, BoundTransformer
from cvrpy.particle.decoder import ParticleDecoder
from cvrpy.particle.factory import ParticleFactory

instance = CVRPInstance.from_file('data/cvrp_castanhal.json')
print(f'Número de clientes: {len(instance.deliveries)}')

particle = ParticleFactory.create_for(instance, sum, num_vehicles=6)
solution = ParticleDecoder.decode(particle, instance)

print(particle)


Número de clientes: 35
Prioridade: [18, 1, 15, 26, 7, 16, 32, 19, 14, 31, 25, 29, 4, 2, 5, 6, 0, 13, 12, 30, 33, 20, 24, 27, 23, 34, 3, 22, 28, 9, 10, 11, 17, 8, 21]
[0.57780614 0.06553498 0.51296932 0.77899778 0.46265549 0.52233708
 0.54243788 0.09106444 0.91147    0.85871131 0.87311091 0.87386886
 0.60496825 0.58108816 0.24609634 0.07348311 0.18278856 0.88955793
 0.051336   0.20788155 0.6940385  0.99406951 0.77905166 0.76551753
 0.71855747 0.2650258  0.08762816 0.72867764 0.79455999 0.30392646
 0.64160869 0.26314781 0.20286306 0.664875   0.77808677 0.57245981
 0.29169727 0.22891258 0.07242627 0.10019354 0.65153018 0.72814658
 0.86137594 0.93675489 0.08009493 0.93852811 0.59557533] => 24.647494
